In [1]:
import pandas as pd
import os
!pip install pykrige

In [2]:
import numpy as np
import pandas as pd
from pykrige.ok import OrdinaryKriging


In [3]:
nlags=20

In [4]:
import pandas as pd
from pykrige.ok import OrdinaryKriging
import numpy as np

def estimate_pollutant(df, date_column, nlags=20):
    # Create a DataFrame with only latitude, longitude, and the specified date column
    df1 = df[['longitude', 'latitude', date_column]]

    # Create a DataFrame with latitude and longitude where the specified date column's values are NaN
    nan_locations_df = df1[df1[date_column].isna()][['longitude', 'latitude']]

    # Create a DataFrame with latitude, longitude, and the specified date column excluding NaN values
    pollutant_df = df1.dropna(subset=[date_column])[['longitude', 'latitude', date_column]]

    # Prepare the data for Ordinary Kriging
    lons = np.array(pollutant_df['longitude'])
    lats = np.array(pollutant_df['latitude'])
    zdata = np.array(pollutant_df[date_column])

    # Perform Ordinary Kriging
    OK = OrdinaryKriging(lons, lats, zdata, variogram_model='linear', verbose=True, enable_plotting=False, nlags=nlags)
    grid_lon = np.array(nan_locations_df["longitude"])
    grid_lat = np.array(nan_locations_df["latitude"])
    z1, ss1 = OK.execute('grid', grid_lon, grid_lat)

    # Fill in NaN values in the specified date column with the interpolated values
    nan_locations_df[date_column] = z1[0]

    # Merge the two DataFrames based on their indices
    merged_df = pd.concat([pollutant_df, nan_locations_df], ignore_index=False)

    # Sort the DataFrame by ascending index
    merged_df = merged_df.sort_index(ascending=True)

    return merged_df

In [5]:
# df should be the DataFrame with columns containing NaN values
df = pd.read_csv("/home/sysadm/Downloads/sentinal5p_O3/Chosen_state_2023_sep_merge_sentinel5p_O3/Delhi_sep_O3.csv")

# Get the list of columns that have NaN values
columns_with_nan = df.columns[df.isna().any()].tolist()

# Create an empty dictionary to store the estimated AOD DataFrames
estimated_data = {}

# Iterate through the columns with NaN values and apply estimate_aod
for date_column in columns_with_nan:
    estimated_data[date_column] = estimate_pollutant(df, date_column)

# Create a list of columns to keep in the estimated_df
columns_to_keep = ['longitude', 'latitude'] + columns_with_nan

# Select only the desired columns from the estimated_df
estimated_df = pd.concat([estimated_data[date_column] for date_column in columns_with_nan], axis=1)
estimated_df = estimated_df[columns_to_keep]

# Drop the duplicate 'latitude' and 'longitude' columns
estimated_df = estimated_df.loc[:,~estimated_df.columns.duplicated()]

# Create a DataFrame 'original_df' containing the original columns without NaN values
original_columns = df.columns.difference(columns_with_nan).tolist()
original_df = df[original_columns]
# Merge the estimated_df and original_df based on 'latitude' and 'longitude'
result_df = original_df.merge(estimated_df, on=['longitude', 'latitude'], how = 'inner')
result_df.to_csv("/home/sysadm/Downloads/sentinal5p_O3/After_krigging_2023_sep_sentinal5p_O3/Delhi_2023_sep_O3_final.csv",index = False)

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1.1494571086452862e-05
Nugget: 6.804406911397847e-06 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 2.2375064345501785e-05
Nugget: 8.392282801575066e-06 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 5.7543601373354436e-05
Nugget: 5.16712773500694e-06 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1.790726681497796e-05
Nugget: 1.0016289048993503e-05 

C

In [6]:
result_df

,latitude,longitude,2023-09-01,2023-09-02,2023-09-03,2023-09-04,2023-09-05,2023-09-06,2023-09-07,2023-09-08,...,2023-09-20,2023-09-21,2023-09-22,2023-09-23,2023-09-24,2023-09-25,2023-09-26,2023-09-27,2023-09-28,2023-09-29
0,28.408461,76.832916,0.125986,0.142418,0.126548,0.127528,0.127284,0.126900,0.131158,0.135261,...,0.121873,0.120363,0.115495,0.117892,0.128985,0.128337,0.124863,0.136919,0.122289,0.128734
1,28.422062,76.832916,0.120517,0.141562,0.126140,0.127264,0.129493,0.130207,0.131158,0.141058,...,0.128531,0.123077,0.130676,0.114506,0.128005,0.121721,0.120243,0.123803,0.119138,0.126502
2,28.435664,76.832916,0.124215,0.139049,0.126140,0.125208,0.129944,0.126845,0.134885,0.137480,...,0.116072,0.125746,0.129509,0.119344,0.123373,0.129341,0.125961,0.139009,0.122333,0.125341
3,28.449265,76.832916,0.119887,0.133765,0.126140,0.122591,0.128334,0.128756,0.131158,0.136701,...,0.119204,0.127821,0.124082,0.119617,0.126717,0.121876,0.124648,0.141504,0.121596,0.125359
4,28.462866,76.832916,0.125019,0.138203,0.126140,0.125208,0.124399,0.130291,0.131158,0.140669,...,0.117697,0.120900,0.127087,0.119888,0.125089,0.121228,0.120738,0.134652,0.119416,0.134201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,28.830106,77.328716,0.119874,0.133678,0.126699,0.129563,0.126861,0.129419,0.130573,0.138368,...,0.120791,0.127305,0.128453,0.114047,0.130193,0.123029,0.122552,0.134943,0.122008,0.124609
1364,28.843708,77.328716,0.117379,0.140109,0.124713,0.126033,0.130987,0.130863,0.136028,0.135003,...,0.115556,0.127703,0.123385,0.115382,0.130221,0.130274,0.122413,0.139684,0.118435,0.137372
1365,28.857309,77.328716,0.118259,0.137078,0.127529,0.123623,0.129577,0.132396,0.117384,0.133180,...,0.119282,0.122581,0.113962,0.117111,0.132674,0.131606,0.128851,0.123779,0.122192,0.124905
1366,28.870910,77.328716,0.126669,0.137791,0.126699,0.122260,0.129271,0.132048,0.130573,0.135218,...,0.128597,0.123782,0.115818,0.116175,0.135486,0.129693,0.124002,0.138589,0.117594,0.126996


In [7]:
# df should be the DataFrame with columns containing NaN values
df = pd.read_csv("/home/sysadm/Downloads/sentinal5p_O3/Chosen_state_2023_sep_merge_sentinel5p_O3/WB_sep_O3.csv")

# Get the list of columns that have NaN values
columns_with_nan = df.columns[df.isna().any()].tolist()

# Create an empty dictionary to store the estimated AOD DataFrames
estimated_data = {}

# Iterate through the columns with NaN values and apply estimate_aod
for date_column in columns_with_nan:
    estimated_data[date_column] = estimate_pollutant(df, date_column)

# Create a list of columns to keep in the estimated_df
columns_to_keep = ['longitude', 'latitude'] + columns_with_nan

# Select only the desired columns from the estimated_df
estimated_df = pd.concat([estimated_data[date_column] for date_column in columns_with_nan], axis=1)
estimated_df = estimated_df[columns_to_keep]

# Drop the duplicate 'latitude' and 'longitude' columns
estimated_df = estimated_df.loc[:,~estimated_df.columns.duplicated()]

# Create a DataFrame 'original_df' containing the original columns without NaN values
original_columns = df.columns.difference(columns_with_nan).tolist()
original_df = df[original_columns]
# Merge the estimated_df and original_df based on 'latitude' and 'longitude'
result_df = original_df.merge(estimated_df, on=['longitude', 'latitude'], how = 'inner')
result_df.to_csv("/home/sysadm/Downloads/sentinal5p_O3/After_krigging_2023_sep_sentinal5p_O3/WB_2023_sep_O3_final.csv",index = False)

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 7.678793525917587e-07
Nugget: 4.9857724623723625e-06 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1.1560956677146316e-06
Nugget: 1.5724841427783627e-05 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 2.8428793257392503e-06
Nugget: 3.482656360248178e-05 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1.8554976165087308e-06
Nugget: 2.977226859718278e-05 


In [9]:
result_df

,latitude,longitude,2023-09-01,2023-09-02,2023-09-03,2023-09-04,2023-09-05,2023-09-06,2023-09-07,2023-09-08,...,2023-09-20,2023-09-21,2023-09-22,2023-09-23,2023-09-24,2023-09-25,2023-09-26,2023-09-27,2023-09-28,2023-09-29
0,21.539446,85.826363,0.121348,0.136622,0.124030,0.128609,0.128139,0.127809,0.136203,0.140493,...,0.116433,0.124778,0.115053,0.119522,0.130302,0.120607,0.122136,0.137705,0.119024,0.126997
1,21.660331,85.826363,0.121348,0.139227,0.123850,0.126119,0.130576,0.128449,0.115164,0.139016,...,0.132266,0.123577,0.122390,0.115447,0.127622,0.130108,0.121094,0.135295,0.120905,0.137943
2,21.781215,85.826363,0.119590,0.139227,0.122157,0.126527,0.127403,0.126119,0.131381,0.141127,...,0.132191,0.126507,0.119411,0.120017,0.125884,0.129418,0.123169,0.136052,0.119440,0.124917
3,21.902100,85.826363,0.122099,0.136620,0.126013,0.120409,0.132000,0.131614,0.137778,0.116278,...,0.127439,0.122679,0.116786,0.119549,0.128108,0.120606,0.120442,0.119038,0.116698,0.122688
4,22.022985,85.826363,0.123944,0.139371,0.121661,0.123362,0.127143,0.132549,0.133671,0.141058,...,0.123682,0.124237,0.130133,0.123488,0.126920,0.119836,0.120365,0.127351,0.123969,0.139084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1627,26.737492,89.786363,0.119996,0.139773,0.120074,0.120800,0.144528,0.140662,0.119232,0.116902,...,0.116567,0.131233,0.111215,0.124140,0.121596,0.120380,0.119616,0.123168,0.119277,0.125572
1628,26.858377,89.786363,0.119996,0.138527,0.140722,0.135709,0.137333,0.140632,0.137195,0.139835,...,0.128919,0.131988,0.111167,0.118456,0.137094,0.123399,0.122259,0.124927,0.122601,0.117637
1629,26.979262,89.786363,0.123394,0.135008,0.125009,0.135709,0.132695,0.139845,0.122387,0.141548,...,0.115831,0.126593,0.119552,0.118133,0.122333,0.118827,0.119499,0.143026,0.120722,0.139250
1630,27.100146,89.786363,0.119996,0.130122,0.122043,0.135709,0.133452,0.141089,0.137195,0.154343,...,0.130753,0.135256,0.115055,0.124444,0.132846,0.119744,0.133995,0.142723,0.123672,0.139833


In [11]:
# df should be the DataFrame with columns containing NaN values
df = pd.read_csv("/home/sysadm/Downloads/sentinal5p_O3/Chosen_state_2023_sep_merge_sentinel5p_O3/karnataka_sep_O3.csv")

# Get the list of columns that have NaN values
columns_with_nan = df.columns[df.isna().any()].tolist()

# Create an empty dictionary to store the estimated AOD DataFrames
estimated_data = {}

# Iterate through the columns with NaN values and apply estimate_aod
for date_column in columns_with_nan:
    estimated_data[date_column] = estimate_pollutant(df, date_column)

# Create a list of columns to keep in the estimated_df
columns_to_keep = ['longitude', 'latitude'] + columns_with_nan

# Select only the desired columns from the estimated_df
estimated_df = pd.concat([estimated_data[date_column] for date_column in columns_with_nan], axis=1)
estimated_df = estimated_df[columns_to_keep]

# Drop the duplicate 'latitude' and 'longitude' columns
estimated_df = estimated_df.loc[:,~estimated_df.columns.duplicated()]

# Create a DataFrame 'original_df' containing the original columns without NaN values
original_columns = df.columns.difference(columns_with_nan).tolist()
original_df = df[original_columns]
# Merge the estimated_df and original_df based on 'latitude' and 'longitude'
result_df = original_df.merge(estimated_df, on=['longitude', 'latitude'], how = 'inner')
result_df.to_csv("/home/sysadm/Downloads/sentinal5p_O3/After_krigging_2023_sep_sentinal5p_O3/Karnataka_2023_sep_O3_final.csv",index = False)

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 2.2284217286506e-06
Nugget: 1.6062699472567743e-05 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1.2341217719892022e-06
Nugget: 1.9326663044251804e-05 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 3.6245970197415995e-07
Nugget: 9.231498718667875e-06 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 4.727604227080005e-07
Nugget: 7.851791549986902e-06 

Ca

In [12]:
result_df

,latitude,longitude,2023-09-01,2023-09-02,2023-09-03,2023-09-04,2023-09-05,2023-09-06,2023-09-07,2023-09-08,...,2023-09-20,2023-09-21,2023-09-22,2023-09-23,2023-09-24,2023-09-25,2023-09-26,2023-09-27,2023-09-28,2023-09-29
0,11.574480,74.054306,0.130120,0.126338,0.121038,0.118603,0.130045,0.123825,0.128427,0.125927,...,0.146544,0.122131,0.147639,0.132517,0.120906,0.128140,0.121017,0.121701,0.128635,0.126177
1,11.714901,74.054306,0.126670,0.126036,0.121038,0.118486,0.122760,0.119672,0.123031,0.124877,...,0.143153,0.122372,0.169547,0.132093,0.120380,0.126171,0.118558,0.120816,0.128451,0.119542
2,11.855323,74.054306,0.128908,0.125807,0.118945,0.117690,0.125695,0.129860,0.122846,0.126098,...,0.136236,0.124977,0.143693,0.118191,0.120936,0.129186,0.118369,0.124085,0.128241,0.121496
3,11.995744,74.054306,0.117407,0.125091,0.120062,0.117841,0.123128,0.130544,0.128000,0.135598,...,0.133195,0.123823,0.145801,0.137875,0.119525,0.128493,0.128888,0.126193,0.128110,0.122352
4,12.136165,74.054306,0.128061,0.125483,0.118050,0.123898,0.128146,0.129752,0.124178,0.123641,...,0.134574,0.123594,0.150450,0.120754,0.125704,0.122578,0.116039,0.117319,0.131173,0.122895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1645,17.893435,78.534306,0.118396,0.134029,0.126506,0.123407,0.124157,0.125091,0.130929,0.133070,...,0.119235,0.122007,0.126583,0.119860,0.124444,0.127849,0.118944,0.133568,0.119129,0.121961
1646,18.033856,78.534306,0.118514,0.140138,0.126201,0.119599,0.128501,0.127090,0.131926,0.131383,...,0.126058,0.125320,0.140681,0.125690,0.126069,0.125070,0.118667,0.138213,0.126635,0.123351
1647,18.174278,78.534306,0.127020,0.122940,0.123870,0.123380,0.128863,0.128693,0.130425,0.128460,...,0.134085,0.123621,0.127340,0.117787,0.117029,0.123654,0.117179,0.123713,0.122577,0.124705
1648,18.314699,78.534306,0.117235,0.131670,0.123870,0.126337,0.123572,0.125294,0.130583,0.131058,...,0.127096,0.120856,0.139552,0.116581,0.122364,0.118777,0.118523,0.134070,0.119408,0.125635
